<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/c1_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## 무조건 해야하는 부분 1
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main Sources [2,159 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Get:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main amd64 Packages [1,108 kB]
Get:13 http://s

In [2]:
import os
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup as bs

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}

# 전체 과정 사이클

In [3]:
url = "https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [5]:
base_df = pd.read_csv("/content/drive/MyDrive/KB인턴관련/crawl_1_euckr.csv", encoding='euc-kr')
base_df.tail()

,category,lawName,ENG,KOR
68167,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,Article 7 (Demand for Materials Related to Con...,제7조(계약에 관한 자료 등의 요구)\n① 기획재정부장관은 정부조달협정가입국으로부터...
68168,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,Article 8 (Reporting Results of Contracts)\n(1...,제8조(계약실적보고)\n① 특례규정 제25조제1항에 따른 계약실적 보고서의 내용은 ...
68169,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,ADDENDUM <Ordinance of the Ministry of Strateg...,"부칙 <제358호, 2013.8.7.>\n이 규칙은 공포한 날부터 시행한다."
68170,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,ADDENDA <Ordinance of the Ministry of Strategy...,"부칙 <제360호, 2013.9.17.> (전자조달의 이용 및 촉진에 관한 법률 시..."
68171,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,ADDENDUM <Ordinance of the Ministry of Strateg...,"부칙 <제492호, 2015.7.20.>\n이 규칙은 정부조달에 관한 협정 개정 의..."


In [6]:
def wait_driver(xpath, t=10):
    WebDriverWait(driver, t).until(EC.presence_of_element_located((By.XPATH, xpath)))

def tab_move():
    driver.switch_to.window(driver.window_handles[-1])
    ko_url = driver.current_url.replace("ENG", "KOR")
    driver.get(ko_url)
    wait_driver('//*[@id="lawViewContent"]')
    driver.switch_to.frame("lawViewContent")

def tab_close():
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    driver.switch_to.frame("indexFrame")

def parsing_laws():
    lang_pairs = {"eng": [], "kor": []}
    for i, p in enumerate(driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')):
        if i<4: continue
        for line in p.text.split("\n"):
            if i%2==0:
                lang_pairs["eng"].append(line)
            else:
                lang_pairs["kor"].append(line)

In [7]:
driver.get(url)

categories = [f'tree_{i}_a' for i in range(2, 47)]
# category loop
base_df = pd.DataFrame()
for j, c in enumerate(categories):
    category = driver.find_element(By.ID, c)
    cate_list = category.text.split(" ")
    cate_name, num_of_laws = cate_list[1], int(cate_list[2].strip('()'))
    driver.execute_script("arguments[0].click();", category)

    # pages
    driver.switch_to.frame('indexFrame')
    # pages = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.paging > a")
    pages = [f'//*[@id="container"]/div/div[3]/a[{k}]' for k in range(3, 13)]
    page_idx = 0
    complete_num = 0
    print(f"{j+1}. {cate_name} 시작...")
    while True:
    # laws loop
        wait_driver('//*[@id="1"]/td[2]')
        laws = driver.find_elements(By.CSS_SELECTOR, '#list1 > tbody > tr > td > p > a')
        page_parsing_num = 0
        for law in laws:
            law_name = law.text.split('\n')[0]
            law.send_keys(Keys.ENTER)

            tab_move()
            wait_driver('/html')

            # Parsing
            eng, kor = [], []
            contents = driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')
            for i, c in enumerate(contents[4:-2]):
                if i%2==0:
                    eng.append(c.text)
                else:
                    kor.append(c.text)
            if len(eng)!=len(kor):
                raise Exception(f"Different Length ENG{len(eng)}, KOR{len(kor)}")
            else:
                df = pd.DataFrame({"category":[cate_name]*len(eng), "lawName":[law_name]*len(eng), "ENG":eng, "KOR":kor})
                page_parsing_num+=df.shape[0]
                # print(f"{law_name} 파싱 완료 : {len(eng)} 개")

            # fininsh parsing
            tab_close()
            base_df = pd.concat([base_df, df], axis=0)
        print(f"\t{page_idx+1}페이지 파싱 완료: {page_parsing_num}")
        complete_num += len(laws)
        if complete_num < num_of_laws:
            driver.find_element(By.XPATH, pages[page_idx]).click()
            page_idx+=1
        else:
            break
    driver.get(url)
    wait_driver('//*[@id="tree_1"]')
driver.quit()


1. 헌법 시작...
	1페이지 파싱 완료: 876
	2페이지 파싱 완료: 420
2. 국회 시작...
	1페이지 파싱 완료: 537
3. 선거ㆍ정당 시작...
	1페이지 파싱 완료: 1095
4. 행정일반 시작...
	1페이지 파싱 완료: 1165
	2페이지 파싱 완료: 1070
	3페이지 파싱 완료: 407
5. 국가공무원 시작...
	1페이지 파싱 완료: 1959
	2페이지 파싱 완료: 851
6. 법원 시작...
	1페이지 파싱 완료: 486
7. 법무 시작...
	1페이지 파싱 완료: 1050
	2페이지 파싱 완료: 1452
	3페이지 파싱 완료: 9
8. 민사법 시작...
	1페이지 파싱 완료: 982
	2페이지 파싱 완료: 2802
	3페이지 파싱 완료: 2715
	4페이지 파싱 완료: 685
	5페이지 파싱 완료: 1561
9. 형사법 시작...
	1페이지 파싱 완료: 615
	2페이지 파싱 완료: 863
	3페이지 파싱 완료: 875
	4페이지 파싱 완료: 1277
	5페이지 파싱 완료: 1618
10. 지방제도 시작...
	1페이지 파싱 완료: 1363
	2페이지 파싱 완료: 2069
	3페이지 파싱 완료: 2647
	4페이지 파싱 완료: 690
11. 경찰 시작...
	1페이지 파싱 완료: 1155
	2페이지 파싱 완료: 852
12. 민방위ㆍ소방 시작...
	1페이지 파싱 완료: 1262
	2페이지 파싱 완료: 1700
	3페이지 파싱 완료: 61
13. 군사 시작...
	1페이지 파싱 완료: 573
	2페이지 파싱 완료: 1594
	3페이지 파싱 완료: 1369
	4페이지 파싱 완료: 600
14. 병무 시작...
	1페이지 파싱 완료: 932
15. 국가보훈 시작...
	1페이지 파싱 완료: 1724
	2페이지 파싱 완료: 384
16. 교육ㆍ학술 시작...
	1페이지 파싱 완료: 1295
	2페이지 파싱 완료: 1044
	3페이지 파싱 완료: 1093
	4페이지 파싱 완료: 913
	5페이지 파싱 완료: 1841
	6페이지 파싱 완료

In [8]:
base_df.to_csv('/content/drive/MyDrive/KB인턴관련/crawl_1706.csv', index=False, encoding='euc-kr')